### Import Needed Libraries

In [10]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [11]:
# import needed libraries
import tensorflow as tf
import numpy as np
# using opencv for feature extraction
import cv2 as cv
import os
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

### Reading Data From Google Drive


In [12]:
# unpacking data
from google.colab import drive
drive.mount('/content/drive')
train_path='/content/drive/My Drive/cv second project/Training'
test_path='/content/drive/My Drive/cv second project/Test'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Extracting Features From Images

In [13]:
classes = sorted(os.listdir(train_path)) # same for test path
# extract features from images
def get_features_from_colors(rgb):
  sum_red = 0
  sum_green = 0
  sum_blue = 0
  count_red = 0
  count_green = 0
  count_blue = 0
  for r in rgb:
    for c in r:
      sum_red += c[0]
      sum_green += c[1]
      sum_blue += c[2]
      count_red += 1
      count_green += 1
      count_blue += 1
  return [sum_red / count_red , sum_green / count_green , sum_blue / count_blue]
def get_features_from_binary(binary , features):
  kernel = np.ones((5,5),np.uint8)
  # closing is used to close holes
  closing = cv.morphologyEx(binary, cv.MORPH_CLOSE, kernel)
  # extracting boundary
  erosion = cv.erode(closing,kernel,iterations = 1)
  boundary = closing - erosion
  boundary_length = 0
  binary_area = 0
  for b in boundary:
    for b2 in b:
      if b2 != 0:
        boundary_length += 1
  features.append(boundary_length)
  # getting area
  for b in binary:
    for b2 in b:
      if b2 != 0:
        binary_area += 1
  features.append(binary_area)
  return features
def get_features_from_gray(gray , features):
  three_rows = [gray[20] , gray[50] , gray[80]]
  pca = PCA(2)
  transformed = pca.fit_transform(three_rows)
  for t in transformed:
    for c in t:
      features.append(c)
  return features

def extract_features(img):
  rgb = cv.cvtColor(img,cv.COLOR_BGR2RGB)
  features = get_features_from_colors(rgb)
  gray = cv.cvtColor(img , cv.COLOR_BGR2GRAY)
  # features = get_features_from_gray(gray , features)
  # convert to binary for extract boundary
  thresh = 25
  binary = cv.threshold(gray, thresh, 255, cv.THRESH_BINARY)[1]
  features = get_features_from_binary(binary , features)
  return features
# reading all images from directories 
def load_images_from(folder):
  images = []
  labels = []
  label = 0
  for foldername in classes:
    for filename in os.listdir(os.path.join(folder , foldername)):
      img = cv.imread(os.path.join(folder,foldername,filename))
      if img is not None:
          features = extract_features(img)
          images.append(np.array(features))
          labels.append(label)
    label+=1
    print(label , ' label reading data passed')
  return images , labels

In [14]:
# running functions above
x , y = load_images_from(train_path)
x_test , y_test = load_images_from(test_path)

1  label reading data passed
2  label reading data passed
3  label reading data passed
4  label reading data passed
5  label reading data passed
6  label reading data passed
7  label reading data passed
8  label reading data passed
9  label reading data passed
10  label reading data passed
11  label reading data passed
12  label reading data passed
13  label reading data passed
14  label reading data passed
15  label reading data passed
16  label reading data passed
17  label reading data passed
18  label reading data passed
19  label reading data passed
20  label reading data passed
21  label reading data passed
22  label reading data passed
23  label reading data passed
24  label reading data passed
25  label reading data passed
26  label reading data passed
27  label reading data passed
28  label reading data passed
29  label reading data passed
30  label reading data passed
31  label reading data passed
32  label reading data passed
33  label reading data passed
34  label reading d

In [15]:
# normalizing data
x_n = normalize(x)
x_test_n = normalize(x_test)

### Fitting KNN Classifier

In [16]:
# fitting KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier()
knn_clf.fit(x_n,y)
y_pred=knn_clf.predict(x_test_n)

### Evaluating Results

In [17]:
# calculating metrics
accuracy = metrics.accuracy_score(y_test , y_pred)
f1_score = metrics.f1_score(y_test , y_pred , average='weighted')
precision = metrics.precision_score(y_test , y_pred , average='weighted')
recall = metrics.recall_score(y_test , y_pred , average='weighted')

In [18]:
#printing results
print('accuracy of svm is : ' , accuracy * 100 , '%')
print('f1 score of svm is : ' , f1_score * 100 , '%')
print('precision of svm is : ' , precision * 100 , '%')
print('recall of svm is : ' , recall * 100  , '%')

accuracy of svm is :  82.2800317071413 %
f1 score of svm is :  82.04981607287793 %
precision of svm is :  83.55900765611476 %
recall of svm is :  82.2800317071413 %
